In [1]:
import numpy as np
import pandas as pd

In [2]:
# Read NPI Canada CSV (U of T group)
# df = pd.read_csv("npi_canada.csv",parse_dates=['start_date', 'end_date']).drop(columns='Unnamed: 0')
df = pd.read_csv("https://raw.githubusercontent.com/jajsmith/COVID19NonPharmaceuticalInterventions/master/npi_canada.csv",
                   parse_dates=['start_date', 'end_date']).drop(columns='Unnamed: 0')

# If the region field is blank, it applies to all Canada
# May want to leave this null?
df["region"].fillna('All', inplace=True)

# If subregion blank, it applies to all cities
# May want to leave this null?
df["subregion"].fillna('All', inplace=True)

# Fix minor typos
df= df.replace({'country' : { 'Canda' : 'Canada', 'Canada ' : 'Canada'}})

# Should not have null values in intervention category (?)
df.intervention_category.fillna('Unclassified', inplace=True)

In [18]:
df.oxford_government_response_category.unique()

array(['H2 Testing policy', nan, 'C3 Cancel public events',
       'H1 Public Info Campaigns', 'C8 International Travel Controls',
       'C1 School Closing', 'H4 Emergency investment in health care',
       'C2 Workplace Closures', 'E3 Fiscal measures',
       'C7 Restrictions on internal movements', 'H3 Contact tracing',
       'C5 Close public transport', 'E1 Income Support',
       'H5 Investment in vaccines',
       'E2 Debt / Contract Relief for Households',
       'E4 Support for Other Countries',
       'C4 Public Gathering Restrictions'], dtype=object)

In [3]:
# When did the dataset start tracking?
start_tracking_date = df.start_date.unique().min()
start_tracking_date

numpy.datetime64('2020-01-07T00:00:00.000000000')

In [4]:
# When's the latest update to the datset?
latest_tracking_date = df.start_date.unique().max()
latest_tracking_date

numpy.datetime64('2020-05-03T00:00:00.000000000')

In [5]:
# Get a date range from the start to end. For each location, one row for each day.
daterange = pd.date_range(start=start_tracking_date,end=latest_tracking_date)
daterange

DatetimeIndex(['2020-01-07', '2020-01-08', '2020-01-09', '2020-01-10',
               '2020-01-11', '2020-01-12', '2020-01-13', '2020-01-14',
               '2020-01-15', '2020-01-16',
               ...
               '2020-04-24', '2020-04-25', '2020-04-26', '2020-04-27',
               '2020-04-28', '2020-04-29', '2020-04-30', '2020-05-01',
               '2020-05-02', '2020-05-03'],
              dtype='datetime64[ns]', length=118, freq='D')

In [6]:
# What are the unique places (country/region/subregion)?
df_places = df[['country', 'region', 'subregion']].drop_duplicates().sort_values(['region', 'subregion'])

# Make list of unique places, condensing name with underscore separation
unique_places = df_places[['country', 'region', 'subregion']].apply(lambda row: '_'.join(row.values.astype(str)), axis=1).values.tolist()
unique_places[0:5]

['Canada_Alberta_All',
 'Canada_Alberta_Calgary',
 'Canada_Alberta_Edmonton',
 'Canada_All_All',
 'nan_All_All']

In [7]:
# Setting columns for the empty dataframe
ox_categories = ['H2 Testing policy', 'C3 Cancel public events',
       'H1 Public Info Campaigns', 'C8 International Travel Controls',
       'C1 School Closing', 'H4 Emergency investment in health care',
       'C2 Workplace Closures', 'E3 Fiscal measures',
       'C7 Restrictions on internal movements', 'H3 Contact tracing',
       'C5 Close public transport', 'E1 Income Support',
       'H5 Investment in vaccines',
       'E2 Debt / Contract Relief for Households',
       'E4 Support for Other Countries',
       'C4 Public Gathering Restrictions']

# ox_categories_with_flags = []
# [ox_categories_with_flags.extend([i, f'{i} flag']) for i in ox_categories]

other_cols_template = [
    'flag',
    'source_url', 
    'source_organization', 
    'source_organization_2', 
    'source_category', 
    'source_title', 
    'source_full_text', 
    'note',
]
ss = [cat.split()[0] for cat in ox_categories]
other_cols_all = []
for s in ss:
    for col in other_cols_template:
        other_cols_all.append(f'{s}_{col}')

In [8]:
# Make empty dataframe with all the neccesary columns and rows (dates)
dff = pd.DataFrame( {
    "name" : unique_places, 
    "days" : [daterange for i in range(len(unique_places))]
})

result = pd.DataFrame([(d, tup.name) for tup in dff.itertuples() for d in tup.days]).rename(columns={0:'date',1:'comboName'})
result[['country','region', 'subregion']] = result.comboName.str.split("_",expand=True)
result = result.drop('comboName', 1)
result = pd.concat([result,pd.DataFrame(columns=ox_categories+other_cols_all)], sort=False)
result.head()

,date,country,region,subregion,H2 Testing policy,C3 Cancel public events,H1 Public Info Campaigns,C8 International Travel Controls,C1 School Closing,H4 Emergency investment in health care,...,E4_source_full_text,E4_note,C4_flag,C4_source_url,C4_source_organization,C4_source_organization_2,C4_source_category,C4_source_title,C4_source_full_text,C4_note
0,2020-01-07,Canada,Alberta,All,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-01-08,Canada,Alberta,All,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-01-09,Canada,Alberta,All,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-01-10,Canada,Alberta,All,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-01-11,Canada,Alberta,All,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
result.columns[result.columns.str.startswith('C6')]

Index([], dtype='object')

In [10]:
def update_master(r, field):
    # Unpack the row into variables
    start, end = r['start_date'], r['end_date']
    
    # Most end_date values are empty; assign these to most recent day
    end = daterange[-1] if pd.isnull(end) else end
    
    # Correcting for a data entry mistake
    if start > end:
        return
    
    country, region, subregion = r['country'], r['region'], r['subregion']
    
    ox_gov_resp_cat = r['oxford_government_response_category']
    ox_score = r[field]
    ox_gov_resp_cat_short = ox_gov_resp_cat[:2]
    
    geog_col = f'{ox_gov_resp_cat_short}_flag'
    geog_score = r['oxford_geographic_target_code']

    # Identify rows in the date range for the unique place.
    ids = result[(result['country']==country) & (result['region']==region) & (result['subregion']==subregion) & (result['date']>=start) & (result['date']<=end)][ox_gov_resp_cat].index
    
    # Score is initially null, which we can't increment (would throw error)
    # So, set initial nulls to zero. 
    result.loc[ids, ox_gov_resp_cat] = result.loc[ids, ox_gov_resp_cat].fillna(0)
    
    # Assign the score
    result.loc[ids, ox_gov_resp_cat] = ox_score
    
    # If there is a geographic targetting score, increment value 
    # (first checking for nulls and setting nulls to 0)
    geog_target_code_present = not pd.isnull(geog_score)
    if geog_target_code_present:
        result.loc[ids, geog_col] = result.loc[ids, geog_col].fillna(0)
        result.loc[ids, geog_col] = geog_score
    
    result.loc[ids[0], f'{ox_gov_resp_cat_short}_source_url'] = r['source_url']
    result.loc[ids[0], f'{ox_gov_resp_cat_short}_source_organization'] = r['source_organization']
    result.loc[ids[0], f'{ox_gov_resp_cat_short}_source_organization_2'] = r['source_organization_2']
    result.loc[ids[0], f'{ox_gov_resp_cat_short}_source_category'] = r['source_category']
    result.loc[ids[0], f'{ox_gov_resp_cat_short}_source_title'] = r['source_title']
    result.loc[ids[0], f'{ox_gov_resp_cat_short}_source_full_text'] = r['source_full_text']
    result.loc[ids[0], f'{ox_gov_resp_cat_short}_note'] = r['note']

In [11]:
df_with_oxford_category = df[~df['oxford_government_response_category'].isnull()]
df_with_oxford_category.head(2)

,start_date,end_date,country,region,subregion,intervention_summary,intervention_category,target_population_category,enforcement_category,oxford_government_response_category,...,source_category,source_title,source_full_text,note,end_source_url,end_source_organization,end_source_organization_2,end_source_category,end_source_title,end_source_full_text
0,2020-03-10,NaT,Canada,Northwest Territories,All,Testing symptomatic residents and visitors who...,Testing Definition Change,Returning Travellers,Not Applicable,H2 Testing policy,...,Government Website,Media Statement - Coronavirus Disease (COVID-1...,Media Statement - Coronavirus Disease (COVID-1...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-03-12,NaT,Canada,Northwest Territories,All,Cancellation of Arctic Winter Games,Public event/ meeting cancellation or postpone...,General,Refusal of Entry,C3 Cancel public events,...,Government Website,Paulie Chinna: Arctic Winter Games Ministers' ...,Paulie Chinna: Arctic Winter Games\nYellowknif...,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
entries_with_ox_cat = df_with_oxford_category.shape[0]
entries_without_ox_cat = df[df['oxford_government_response_category'].isnull()].shape[0]
print(f'Entries WITH Oxford categories: {entries_with_ox_cat}')
print(f'Entries WITHOUT Oxford categories: {entries_without_ox_cat}')

Entries WITH Oxford categories: 1430
Entries WITHOUT Oxford categories: 1490


In [13]:
# Assumption: each row has scores for ONE Oxford category (not multiple)
for idx, row in df_with_oxford_category.iterrows():
    # Skip nulls, we're only looking for integers 
    if not pd.isnull(row['oxford_closure_code']):
        update_master(row, 'oxford_closure_code')
    if not pd.isnull(row['oxford_public_info_code']):
        update_master(row, 'oxford_public_info_code')
    if not pd.isnull(row['oxford_travel_code']):
        update_master(row, 'oxford_travel_code')
    if not pd.isnull(row['oxford_testing_code']):
        update_master(row, 'oxford_testing_code')
    if not pd.isnull(row['oxford_tracing_code']):
        update_master(row, 'oxford_tracing_code')

result.head(10)

,date,country,region,subregion,H2 Testing policy,C3 Cancel public events,H1 Public Info Campaigns,C8 International Travel Controls,C1 School Closing,H4 Emergency investment in health care,...,E4_source_full_text,E4_note,C4_flag,C4_source_url,C4_source_organization,C4_source_organization_2,C4_source_category,C4_source_title,C4_source_full_text,C4_note
0,2020-01-07,Canada,Alberta,All,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-01-08,Canada,Alberta,All,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-01-09,Canada,Alberta,All,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-01-10,Canada,Alberta,All,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-01-11,Canada,Alberta,All,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2020-01-12,Canada,Alberta,All,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2020-01-13,Canada,Alberta,All,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2020-01-14,Canada,Alberta,All,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2020-01-15,Canada,Alberta,All,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2020-01-16,Canada,Alberta,All,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
rearranged_cols = result.columns[0:4].to_list() + sorted(result.columns[4:].to_list())
result = result[rearranged_cols]
result.to_csv('result_new.csv', index=False)

In [15]:
result.head(10)

,date,country,region,subregion,C1 School Closing,C1_flag,C1_note,C1_source_category,C1_source_full_text,C1_source_organization,...,H4_source_url,H5 Investment in vaccines,H5_flag,H5_note,H5_source_category,H5_source_full_text,H5_source_organization,H5_source_organization_2,H5_source_title,H5_source_url
0,2020-01-07,Canada,Alberta,All,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-01-08,Canada,Alberta,All,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-01-09,Canada,Alberta,All,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-01-10,Canada,Alberta,All,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-01-11,Canada,Alberta,All,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2020-01-12,Canada,Alberta,All,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2020-01-13,Canada,Alberta,All,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2020-01-14,Canada,Alberta,All,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2020-01-15,Canada,Alberta,All,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2020-01-16,Canada,Alberta,All,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
